In [1]:
import numpy as np
import pandas as pd
from template import *
from itertools import chain, combinations
from datetime import datetime

C:\Users\CLILI\AppData\Local\Temp\ipykernel_18664\4125468805.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Exercise 1

Se encuentra en el archivo adjunto

# Exercise 2

## Exercise 2.1

In [2]:
def eclat(db, minsup):
    start_time=datetime.now()
    time_format="%H:%M:%S:%f"
    temporal_items_list = []
    for t in db:
        temporal_items_list.extend(t)
    temporal_items_list = list(sorted(set(temporal_items_list)))
    item_list = temporal_items_list.copy()
    item_list_transactions = {item: [] for item in temporal_items_list}
    for transaction_index, t in enumerate(db):
        for transaction_item in t:
            item_list_transactions[(transaction_item)].append(transaction_index)
    itemset_supports = [([item], item_list_transactions[item]) for item in item_list]
    items_temporal=itemset_supports.copy()
    temporal_items_list = item_list.copy()
    
    for itemset in itemset_supports:
        if len(itemset[1]) < minsup:
            temporal_items_list.remove(itemset[0][0])
            items_temporal.remove(itemset)
    itemset_supports=items_temporal.copy()
    item_list = temporal_items_list.copy()
    
    final_supports_list=supportsNextLevel(itemset_supports,itemset_supports, item_list, minsup)
    final_supports=[(item,len(transactions)) for item,transactions in final_supports_list]
    final_time=datetime.now()
    execution_time=(final_time-start_time)
    print(f'Runtime: {execution_time}\n    Start Time: {start_time.strftime(time_format)}\n    Finish Time: {final_time.strftime(time_format)}')
    return final_supports

    
def supportsNextLevel(total_supports, supports, items, minsup, first_time=True):
    for index, itemset in enumerate(items[:-1]):
        level_supports = []
        itemset_derivate_list = []
        items_to_combine = items[index+1:]
        for other_itemset in items_to_combine:
            if first_time:
                new_itemset_support = findSupport(supports, [itemset], [other_itemset])
                new_itemset_list = sorted(list(set([itemset, other_itemset])))
            else:
                new_list = []
                new_list.extend(itemset)
                new_list.extend(other_itemset)
                new_itemset_support = findSupport(supports, itemset, other_itemset)
                new_itemset_list = sorted(list(set(new_list)))
            if len(new_itemset_support) >= minsup:
                level_supports.append((new_itemset_list, new_itemset_support))
                itemset_derivate_list.append(new_itemset_list)   
        total_supports.extend(level_supports)
        if len(level_supports) > 1 and level_supports:
            supportsNextLevel(total_supports, level_supports, itemset_derivate_list, minsup, False)
    if first_time:
        return total_supports


def findSupport(supports, first_itemset, second_itemset):
    for support in supports:
        if first_itemset == support[0]:
            first_support = support[1]
        if second_itemset == support[0]:
            second_support = support[1]
    return list(set(first_support) & set(second_support))


In [2]:
#Execution With Shop Dataset and 500 Minsups
dbShop= read_database("shop.dat")
itemset_supports = eclat(dbShop, 500)
column_names=["Itemset", "Support"]

NameError: name 'eclat' is not defined

In [4]:
column_names=["Itemset", "Support"]
df_itemset_supports = pd.DataFrame(itemset_supports,columns=column_names)
display(df_itemset_supports)

,Itemset,Support
0,[0],594
1,[1],1535
2,[10],1351
3,[100],1749
4,[104],1158
...,...,...
1068,"[888, 935]",535
1069,"[888, 969]",810
1070,"[914, 984]",546
1071,"[923, 947]",859


## Exercise 2.2

In [5]:
def getStrongRulesFromFrequentSets(fsets, minconf):
   strong_rules = []
   fsets_sets = [fset[0] for fset in fsets]
   fsets_supports = [fset[1] for fset in fsets]

   for i, set in enumerate(fsets_sets):
        if len(set) >= 2:
           A = subsets_non_empty(set)
           while A:
               X = max(A, key=lambda x: fsets_supports[fsets_sets.index(x)])
               A.remove(X)
               index = fsets_sets.index(X)
               sup_z = fsets_supports[i]
               sup_x = fsets_supports[index]
               conf = sup_z / sup_x
               if conf >= minconf:
                   Y = set.copy()
                   for x in X:
                       Y.remove(x)
                   strong_rules.append((X, Y, fsets_supports[i], conf))
               else: 
                  for w in X:
                     if w in A:
                        A.remove(x)                
   return strong_rules

def subsets_non_empty(Z):
   all_subsets = chain.from_iterable(combinations(Z,r) for r in range(1, len(Z) + 1)) 
   non_empty_subsets = [sorted(subset) for subset in all_subsets if subset]
   non_empty_subsets.sort(key=lambda x: (len(x), x))
   non_empty_subsets.pop(-1)
   return non_empty_subsets

def getStrongRulesForDatabase(db, minsup, minconf):
  fsets = eclat(db, minsup)
  strong_rules = getStrongRulesFromFrequentSets(fsets, minconf)
  return strong_rules

In [6]:
rules_shop=getStrongRulesForDatabase(dbShop,500,0.75)

Runtime: 0:00:37.625048
    Start Time: 11:50:53:335212
    Finish Time: 11:51:30:960260


In [7]:
column_names_rules=["Preposition", "Conclusion", "Abs. Support", "Confidence"]
df_rules_shop = pd.DataFrame(rules_shop,columns=column_names_rules)
display(df_rules_shop)

,Preposition,Conclusion,Abs. Support,Confidence
0,"[494, 815]",[105],552,0.883200
1,"[105, 494]",[815],552,0.887460
2,"[105, 815]",[494],552,0.896104
3,"[494, 862]",[105],549,0.762500
4,"[105, 494]",[862],549,0.882637
...,...,...,...,...
959,[842],[803],607,0.807181
960,"[862, 944]",[815],552,0.837633
961,"[815, 944]",[862],552,0.881789
962,"[815, 862]",[944],552,0.897561


## Exercise 2.3

In [8]:
df_filtrado = df_rules_shop[df_rules_shop["Conclusion"].apply(lambda x: len(x) >= 2)]
df_filtrado = df_filtrado.sort_values(by="Conclusion", key=lambda x: x.apply(len), ascending=False).reset_index(drop=True)
display(df_filtrado)

,Preposition,Conclusion,Abs. Support,Confidence
0,[969],"[208, 290, 458, 888]",693,0.816254
1,"[579, 842]","[350, 572, 803]",503,0.838333
2,"[572, 579]","[350, 803, 842]",503,0.836938
3,"[350, 842]","[572, 579, 803]",503,0.836938
4,"[572, 803]","[350, 579, 842]",503,0.832781
...,...,...,...,...
423,"[546, 661]","[217, 947]",572,0.937705
424,"[661, 947]","[217, 546]",572,0.897959
425,"[217, 661]","[546, 947]",572,0.825397
426,"[217, 947]","[546, 661]",572,0.798883


Para el análisis se explicaran las siguientes reglas; regla 0, y para el resto de las reglas se seguirá esa misma lógica:

### Regla 0:

Si un cliente compra el producto [969], entonces es probable que también compre los productos [208, 290, 458, 888]. El soporte es la medida de la frecuencia con la que ocurre un conjunto de elementos juntos en el conjunto de datos. En este caso, el soporte de la regla 0 es 693, lo que significa que el conjunto de productos [969] y [208, 290, 458, 888] se encuentra en 693 transacciones del total de transacciones. Y la confianza es la medida de qué tan probable es que ocurra la conclusión dado que se ha cumplido la preposición, para este caso, en la regla 0, la confianza es de 0.816254, lo que significa que el 81.63% de las veces que se compra [969], también se compran [208, 290, 458, 888].

En resumen, la regla 0 sugiere que hay una asociación fuerte entre la compra de [969] y [208, 290, 458, 888], con un soporte bastante alto (693 transacciones) y también una alta confianza (81.63%).


### Regla 1:

Si un cliente compra los productos [579,842], entonces es probable que también compre los productos [350, 572, 803]. El soporte es la medida de la frecuencia con la que ocurre un conjunto de elementos juntos en el conjunto de datos. En este caso, el soporte de la regla 1 es 503, lo que significa que el conjunto de productos [579,842] y [350, 572, 803] se encuentra en 503 transacciones del total de transacciones. Y la confianza es la medida de qué tan probable es que ocurra la conclusión dado que se ha cumplido la preposición, para este caso, en la regla 1, la confianza es de 0.838333, lo que significa que el 83.83% de las veces que se compra [579,842], también se compran [350, 572, 803].

En resumen, la regla 1 sugiere que hay una asociación fuerte entre la compra de [579,842] y [350, 572, 803], con un soporte alto (503 transacciones) y también una alta confianza (83.83%).

### Regla 2:

Si un cliente compra los productos [572,579], entonces es probable que también compre los productos [350, 803,842]. El soporte es la medida de la frecuencia con la que ocurre un conjunto de elementos juntos en el conjunto de datos. En este caso, el soporte de la regla 2 es 503, lo que significa que el conjunto de productos [572,579] y [350, 803,842] se encuentra en 503 transacciones del total de transacciones. Y la confianza es la medida de qué tan probable es que ocurra la conclusión dado que se ha cumplido la preposición, para este caso, en la regla 2, la confianza es de 0.836938, lo que significa que el 83.69% de las veces que se compra [572,579], también se compran [350, 803,842].

En resumen, la regla 2 sugiere que hay una asociación fuerte entre la compra de [572,579] y [350, 803,842], con un soporte alto (503 transacciones) y también una alta confianza (83.69%).

# Tests

In [9]:
## Unit test
import pandas as pd
dbExample = read_database("example.dat")

def testSyntax(F):
    found_problem = False
    for t in F:
        if type(t) != tuple:
            print("Entry " + str(t) + " in frequent itemset collection F is not a tuple.")
            found_problem = True
        elif len(t) != 2:
            print("Entry " + str(t) + " in frequent itemset collection F is a tuple but of length " + str(len(t)) + " instead of 2.")
            found_problem = True
        elif type(t[0]) != list:
            print("Itemset type in entry " + str(t) + " is " + str(type(t[0])) + " but should be list!")
            found_problem = True
        elif type(t[1]) != int:
            print("Support in entry " + str(t) + " is of type " + str(type(t[1])) + " but should be int!")
            found_problem = True
    print("Syntax check on frequent itemset collection: " + ("FAILED" if found_problem else "OK"))

def testFrequentItemsetCollection(F_exp, F_act):
    testSyntax(F_act)
    length_exp = len(F_exp)
    length_act = len(F_act)
    print("Length of collection: " + str("OK" if length_exp == length_act else "FAILED, expected length " + str(length_exp) + " but saw " + str(length_act)))
    obsolete = [f for f in F_act if not f in F_exp]
    missing = [f for f in F_exp if not f in F_act]
    if obsolete:
        print("Found unexpected entries in F: " + str(obsolete))
    if missing:
        print("Missing entries in F: " + str(missing))
    if not missing and not obsolete:
        print("F seems to be correct.")

F_exp = [(['A'], 4), (['A', 'B'], 4), (['A', 'B', 'D'], 3), (['A', 'B', 'D', 'E'], 3), (['A', 'B', 'E'], 4), (['A', 'D'], 3), (['A', 'D', 'E'], 3), (['A', 'E'], 4), (['B'], 6), (['B', 'C'], 4), (['B', 'C', 'E'], 3), (['B', 'D'], 4), (['B', 'D', 'E'], 3), (['B', 'E'], 5), (['C'], 4), (['C', 'E'], 3), (['D'], 4), (['D', 'E'], 3), (['E'], 5)]

# Test ECLAT
F_act = eclat(dbExample, 3)

testFrequentItemsetCollection(F_exp, F_act)

Runtime: 0:00:00
    Start Time: 11:51:31:120200
    Finish Time: 11:51:31:120200
Syntax check on frequent itemset collection: OK
Length of collection: OK
F seems to be correct.


In [10]:
def testRuleGeneration(R_exp, R_act):

    # test syntax
    found_problem = False
    for r in R_act:
        if type(r) != tuple:
            print("Rule " + str(r) + " is not represented as a tuple!")
            found_problem = True
        if len(r) != 4:
            print("Rule " + str(r) + " does not consist of 4 entries. Should be (premise, conclusion, (absolute) support, confidence)")
            found_problem = True
        if type(r[0]) != list:
            print("Premise of rule " + str(r) + " is of type " + str(type(r[0])) + " but should be list!")
            found_problem = True
        if type(r[1]) != list:
            print("Conclusion of rule " + str(r) + " is of type " + str(type(r[0])) + " but should be list!")
            found_problem = True
    print("Syntax check " + ("FAILED. Skipping rest of the test." if found_problem else "OK"))
    if found_problem:
        return
    
    # semantics test
    length_exp = len(R_exp)
    length_act = len(R_act)
    print("Length of rule set: " + str("OK" if length_exp == length_act else "FAILED, expected length " + str(length_exp) + " but saw " + str(length_act)))
    obsolete = [f for f in R_act if not f in R_exp]
    missing = [f for f in R_exp if not f in R_act]
    if obsolete:
        print("Found unexpected entries in R: " + str(obsolete))
        found_problem = True
    if missing:
        print("Missing entries in R: " + str(missing))
        found_problem = True
    print("Rule set test: " + ("FAILED" if found_problem else "OK"))

R_exp = [(['A'], ['B'], 4, 1.0), (['A', 'D'], ['B'], 3, 1.0), (['B', 'D', 'E'], ['A'], 3, 1.0), (['A', 'D', 'E'], ['B'], 3, 1.0), (['A', 'B', 'D'], ['E'], 3, 1.0), (['D', 'E'], ['A', 'B'], 3, 1.0), (['A', 'D'], ['B', 'E'], 3, 1.0), (['A', 'E'], ['B'], 4, 1.0), (['A', 'B'], ['E'], 4, 1.0), (['A'], ['B', 'E'], 4, 1.0), (['D', 'E'], ['A'], 3, 1.0), (['A', 'D'], ['E'], 3, 1.0), (['A'], ['E'], 4, 1.0), (['C'], ['B'], 4, 1.0), (['C', 'E'], ['B'], 3, 1.0), (['D'], ['B'], 4, 1.0), (['D', 'E'], ['B'], 3, 1.0), (['E'], ['B'], 5, 1.0)]
R_act = getStrongRulesForDatabase(dbExample, 3, 1.0)
testRuleGeneration(R_exp, R_act)

Runtime: 0:00:00
    Start Time: 11:51:31:133810
    Finish Time: 11:51:31:133810
Syntax check OK
Length of rule set: OK
Rule set test: OK
